In [1]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Load datasets
sentiment_df = pd.read_excel("mental_health_datasets .xlsx", sheet_name="Sentiment Analysis")
emotion_df = pd.read_excel("mental_health_datasets .xlsx", sheet_name="Emotional State Classification")

# Prepare sentiment analysis data
X_sentiment = sentiment_df["Text"].values
y_sentiment = sentiment_df["Sentiment"].values
sentiment_encoder = LabelEncoder()
y_sentiment = sentiment_encoder.fit_transform(y_sentiment)

# Prepare emotional state classification data
X_emotion = emotion_df["Text"].values
y_emotion = emotion_df["Emotion"].values
emotion_encoder = LabelEncoder()
y_emotion = emotion_encoder.fit_transform(y_emotion)

# Tokenization
max_words = 5000
max_len = 100
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(np.concatenate((X_sentiment, X_emotion)))

X_sentiment = tokenizer.texts_to_sequences(X_sentiment)
X_sentiment = pad_sequences(X_sentiment, maxlen=max_len)

X_emotion = tokenizer.texts_to_sequences(X_emotion)
X_emotion = pad_sequences(X_emotion, maxlen=max_len)

# Split datasets
X_train_sent, X_test_sent, y_train_sent, y_test_sent = train_test_split(X_sentiment, y_sentiment, test_size=0.2, random_state=42)
X_train_emot, X_test_emot, y_train_emot, y_test_emot = train_test_split(X_emotion, y_emotion, test_size=0.2, random_state=42)

sentiment_model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    LSTM(64),
    Dense(3, activation="softmax")  # 3 classes: Positive, Neutral, Negative
])

sentiment_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
sentiment_model.fit(X_train_sent, y_train_sent, epochs=5, validation_data=(X_test_sent, y_test_sent))
sentiment_model.save("sentiment_model.h5")

C:\Users\vishn\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 404s 158ms/step - accuracy: 0.9649 - loss: 0.0824 - val_accuracy: 1.0000 - val_loss: 7.3214e-06
Epoch 2/5
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 402s 161ms/step - accuracy: 1.0000 - loss: 7.1452e-06 - val_accuracy: 1.0000 - val_loss: 1.5146e-06
Epoch 3/5
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 425s 170ms/step - accuracy: 1.0000 - loss: 1.5960e-06 - val_accuracy: 1.0000 - val_loss: 4.2006e-07
Epoch 4/5
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 361s 144ms/step - accuracy: 1.0000 - loss: 4.5814e-07 - val_accuracy: 1.0000 - val_loss: 1.0245e-07
Epoch 5/5
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 168s 67ms/step - accuracy: 1.0000 - loss: 1.1851e-07 - val_accuracy: 1.0000 - val_loss: 1.5646e-08


In [2]:
emotion_model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    LSTM(64),
    Dense(len(emotion_encoder.classes_), activation="softmax")  # Number of emotion categories
])

emotion_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
emotion_model.fit(X_train_emot, y_train_emot, epochs=5, validation_data=(X_test_emot, y_test_emot))
emotion_model.save("emotion_model.h5")

Epoch 1/5
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 165s 65ms/step - accuracy: 0.9483 - loss: 0.1607 - val_accuracy: 1.0000 - val_loss: 4.0979e-05
Epoch 2/5
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 163s 65ms/step - accuracy: 1.0000 - loss: 3.6649e-05 - val_accuracy: 1.0000 - val_loss: 8.1208e-06
Epoch 3/5
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 166s 66ms/step - accuracy: 1.0000 - loss: 7.9011e-06 - val_accuracy: 1.0000 - val_loss: 2.1655e-06
Epoch 4/5
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 170s 68ms/step - accuracy: 1.0000 - loss: 2.1188e-06 - val_accuracy: 1.0000 - val_loss: 5.9146e-07
Epoch 5/5
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 176s 70ms/step - accuracy: 1.0000 - loss: 5.9993e-07 - val_accuracy: 1.0000 - val_loss: 1.4997e-07
